In [134]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [124]:
ka = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_fraud.csv', on_bad_lines='skip')
dfa = ka.copy()

# Separating X and y
Xa = dfa.drop('fraud_reported', axis=1)
Ya = dfa['fraud_reported']

X_traina, X_testa, y_traina, y_testa = train_test_split(Xa, Ya, test_size=0.25)

In [125]:
#ça devrait donner 1, fraud
a = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_example.csv', on_bad_lines='skip')
a

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1406.91,0,53300,0,1,5,1,1,13020


In [126]:
#ça devrait donner 0, pas de fraud
b = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/example_knn.csv', on_bad_lines='skip')
b

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1442.99,0,0,0,2,21,1,1,2770


In [127]:
b

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1442.99,0,0,0,2,21,1,1,2770


In [141]:
load_clf = pickle.load(open('insurance.pkl', 'rb'))
prediction = load_clf.predict(b)
prediction_proba = load_clf.predict_proba(b)

In [142]:
prediction

array([0], dtype=int64)

In [143]:
prediction_proba

array([[1., 0.]])

In [136]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_traina, y_traina)

KNeighborsClassifier(n_neighbors=3)

In [139]:
knn.predict(b)

array([0], dtype=int64)

In [140]:
knn.predict_proba(b)

array([[1., 0.]])

In [147]:
log = pickle.load(open('insurance_log.pkl', 'rb'))
predlog = log.predict(b)
probalog = log.predict_proba(b)

In [148]:
predlog.item(0)

0

In [149]:
probalog

array([[0.80465356, 0.19534644]])

In [34]:
tree = pickle.load(open('insurance_tree_1.pkl', 'rb'))
predT = tree.predict(b)
probaT = tree.predict_proba(b)

In [35]:
predT

array([0], dtype=int64)

In [36]:
probaT

array([[1., 0.]])

In [37]:
from sklearn.tree import DecisionTreeClassifier

In [38]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [39]:
dtc.predict(b)

array([0], dtype=int64)

In [40]:
dtc.predict_proba(b)

array([[1., 0.]])

In [205]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

st.write("""
# Insurance fraud detection: IMAFA

CEO: Gertaldi Negre Aayar Espicier

This app predicts if a claim is a fraud or not !

[Notebook](https://colab.research.google.com/drive/1w2xC6w6CkbCVCrsTrxsHQffDtwLzGWkS)


Dataset used: [ins_cl_dataset](https://github.com/oualid-ben/data/blob/main/ins_cl_dataset.csv).
""")

st.sidebar.header('User Input Features')

st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_example.csv)
""")

# Collects user input features into dataframe
uploaded_file = st.sidebar.file_uploader("Upload your input CSV file", type=["csv"])
if uploaded_file is not None:
    input_df = pd.read_csv(uploaded_file)
else:
    def user_input_features():
        model  = st.sidebar.selectbox('Model',('KNN', 'Decision Tree'))
        policy_annual_premium  = st.sidebar.number_input('policy annual premium ', 433.33, 2047.59, 1406.91)
        
        umbrella_limit  = st.sidebar.number_input('umbrella limit ', -1000000, 10000000, 0)
        capital_gains  = st.sidebar.number_input('capital gains ', 0, 100500, 53300)
        capital_loss  = st.sidebar.number_input('capital loss ', -111100, 0, 0)
        incident_severity  = st.sidebar.number_input('incident severity ', 0, 3, 1)
        incident_hour_of_the_day  = st.sidebar.number_input('incident hour of the day ', 0, 23, 5)
        number_of_vehicles_involved  = st.sidebar.number_input('number of vehicles involved ', 1,4,1)
        bodily_injuries  = st.sidebar.number_input('bodily injuries ', 0, 2, 1)
        property_claim  = st.sidebar.number_input('property claim ', 0, 23670, 13020)

        data = {'model': model,
                'policy_annual_premium': policy_annual_premium,
                
                'umbrella_limit': umbrella_limit,
                'capital_gains': capital_gains,
                'capital_loss': capital_loss,
                'incident_severity': incident_severity,
                'incident_hour_of_the_day': incident_hour_of_the_day,
                'number_of_vehicles_involved': number_of_vehicles_involved,
                'bodily_injuries': bodily_injuries,
                'property_claim': property_claim,
               }
        features = pd.DataFrame(data, index=[0])
        return features
    input_df = user_input_features()    

# Combines user input features with entire penguins dataset
# This will be useful for the encoding phase
insurance_raw = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data.csv')
opt = input_df.drop(columns=['model'], axis=1)
df = pd.concat([opt ,insurance_raw],axis=0)

df = df[:1] # Selects only the first row (the user input data)

# Displays the user input features
st.subheader('User Input features')

import streamlit as st
import pandas as pd
import numpy as np
import pickle

st.write("""
# Insurance fraud detection: IMAFA

CEO: Gertaldi Negre Aayar Espicier

This app predicts if a claim is a fraud or not !

[Notebook](https://colab.research.google.com/drive/1w2xC6w6CkbCVCrsTrxsHQffDtwLzGWkS)


Dataset used: [ins_cl_dataset](https://github.com/oualid-ben/data/blob/main/ins_cl_dataset.csv).
""")

st.sidebar.header('User Input Features')

st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_example.csv)
""")

# Collects user input features into dataframe
uploaded_file = st.sidebar.file_uploader("Upload your input CSV file", type=["csv"])
if uploaded_file is not None:
    input_df = pd.read_csv(uploaded_file)
else:
    def user_input_features():
        model  = st.sidebar.selectbox('Model',('KNN', 'RandomForest'))
        policy_annual_premium  = st.sidebar.number_input('policy annual premium ', 433.33, 2047.59, 1406.91)
        
        umbrella_limit  = st.sidebar.number_input('umbrella limit ', -1000000, 10000000, 0)
        capital_gains  = st.sidebar.number_input('capital gains ', 0, 100500, 53300)
        capital_loss  = st.sidebar.number_input('capital loss ', -111100, 0, 0)
        incident_severity  = st.sidebar.number_input('incident severity ', 0, 3, 1)
        incident_hour_of_the_day  = st.sidebar.number_input('incident hour of the day ', 0, 23, 5)
        number_of_vehicles_involved  = st.sidebar.number_input('number of vehicles involved ', 1,4,1)
        bodily_injuries  = st.sidebar.number_input('bodily injuries ', 0, 2, 1)
        property_claim  = st.sidebar.number_input('property claim ', 0, 23670, 13020)

        data = {'model': model,
                'policy_annual_premium': policy_annual_premium,
                
                'umbrella_limit': umbrella_limit,
                'capital_gains': capital_gains,
                'capital_loss': capital_loss,
                'incident_severity': incident_severity,
                'incident_hour_of_the_day': incident_hour_of_the_day,
                'number_of_vehicles_involved': number_of_vehicles_involved,
                'bodily_injuries': bodily_injuries,
                'property_claim': property_claim,
               }
        features = pd.DataFrame(data, index=[0])
        return features
    input_df = user_input_features()    

# Combines user input features with entire penguins dataset
# This will be useful for the encoding phase
insurance_raw = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data.csv')
opt = input_df.drop(columns=['model'], axis=1)
df = pd.concat([opt ,insurance_raw],axis=0)

df = df[:1] # Selects only the first row (the user input data)

# Displays the user input features
st.subheader('User Input features')

if uploaded_file is not None:
    st.write(df)
else:
    st.write('Awaiting CSV file to be uploaded. Currently using example input parameters (shown below).')
    st.write(df)

if input_df['model'].iat[0]=='KNN':    
    # Reads in saved classification model
    load_clf = pickle.load(open('insurance.pkl', 'rb'))
    # Apply model to make predictions
    prediction = load_clf.predict(df)
    prediction_proba = load_clf.predict_proba(df)
else:
    data = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_fraud.csv')
    d = {'Minor Damage': 0, 'Major Damage': 1, 'Total Loss': 2, 'Trivial Damage': 3}
    data['incident_severity'] = data['incident_severity'].replace(d)
    d_ = {'N': 0, 'Y': 1}
    data['fraud_reported'] = data['fraud_reported'].replace(d_)
    X = data.drop('fraud_reported', axis=1)
    y = data['fraud_reported']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    prediction = dtc.predict(df)
    prediction_proba= dtc.predict_proba(df)


st.subheader('Prediction')
insurance_species = np.array(['No fraud','Yes, fraud'])
st.write(insurance_species[prediction.item(0)])

st.subheader('Prediction Probability')
st.write(prediction_proba)

print(prediction_proba)

[[0.33333333 0.66666667]]


In [163]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

st.write("""
# Insurance fraud detection: IMAFA

CEO: Gertaldi Negre Aayar Espicier

This app predicts if a claim is a fraud or not !

[Notebook](https://colab.research.google.com/drive/1w2xC6w6CkbCVCrsTrxsHQffDtwLzGWkS)


Dataset used: [ins_cl_dataset](https://github.com/oualid-ben/data/blob/main/ins_cl_dataset.csv).
""")

st.sidebar.header('User Input Features')

st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_example.csv)
""")

# Collects user input features into dataframe
uploaded_file = st.sidebar.file_uploader("Upload your input CSV file", type=["csv"])
if uploaded_file is not None:
    input_df = pd.read_csv(uploaded_file)
else:
    def user_input_features():
        model  = st.sidebar.selectbox('Model',('KNN', 'Decision Tree'))
        policy_annual_premium  = st.sidebar.number_input('policy annual premium ', 433.33, 2047.59, 1406.91)
        
        umbrella_limit  = st.sidebar.number_input('umbrella limit ', -1000000, 10000000, 0)
        capital_gains  = st.sidebar.number_input('capital gains ', 0, 100500, 53300)
        capital_loss  = st.sidebar.number_input('capital loss ', -111100, 0, 0)
        incident_severity  = st.sidebar.number_input('incident severity ', 0, 3, 1)
        incident_hour_of_the_day  = st.sidebar.number_input('incident hour of the day ', 0, 23, 5)
        number_of_vehicles_involved  = st.sidebar.number_input('number of vehicles involved ', 1,4,1)
        bodily_injuries  = st.sidebar.number_input('bodily injuries ', 0, 2, 1)
        property_claim  = st.sidebar.number_input('property claim ', 0, 23670, 13020)

        data = {'model': model,
                'policy_annual_premium': policy_annual_premium,
                
                'umbrella_limit': umbrella_limit,
                'capital_gains': capital_gains,
                'capital_loss': capital_loss,
                'incident_severity': incident_severity,
                'incident_hour_of_the_day': incident_hour_of_the_day,
                'number_of_vehicles_involved': number_of_vehicles_involved,
                'bodily_injuries': bodily_injuries,
                'property_claim': property_claim,
               }
        features = pd.DataFrame(data, index=[0])
        return features
    input_df = user_input_features()    

# Combines user input features with entire penguins dataset
# This will be useful for the encoding phase
insurance_raw = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data.csv')
opt = input_df.drop(columns=['model'], axis=1)
df = pd.concat([opt ,insurance_raw],axis=0)

df = df[:1] # Selects only the first row (the user input data)

# Displays the user input features
st.subheader('User Input features')

if uploaded_file is not None:
    st.write(df)
else:
    st.write('Awaiting CSV file to be uploaded. Currently using example input parameters (shown below).')
    st.write(df)

# Reads in saved classification model
load_clf = pickle.load(open('insurance.pkl', 'rb'))

# Apply model to make predictions
prediction = load_clf.predict(df)
prediction_proba = load_clf.predict_proba(df)


st.subheader('Prediction')
insurance_species = np.array(['No fraud','Yes, fraud'])
st.write(insurance_species[prediction.item(0)])

st.subheader('Prediction Probability')
st.write(prediction_proba)

print(prediction_proba)

[[0.33333333 0.66666667]]
